In [2]:
class Point:
    def __init__(self, x, y, a, b, p):
        self.x = x % p if x is not None else None
        self.y = y % p if y is not None else None
        self.a = a % p if a is not None else None
        self.b = b % p if b is not None else None
        self.p = p

        if self.x is None and self.y is None:
            return  # Point at infinity

        if (self.y ** 2) % p != (self.x ** 3 + a * self.x + b) % p:
            raise ValueError(f"({self.x}, {self.y}) is not on the curve.")
            
    def __str__(self):
        return f"Point({self.x}, {self.y}) on curve y^2 = x^3 + {self.a}x + {self.b} mod {self.p}"
            
    def __get_inf(self):
        return Point(None, None, self.a, self.b, self.p)

    def __add__(self, other):
        if self.p != other.p:
            raise TypeError(f"Points {self}, {other} are not in the same field")
        
        if self.a != other.a or self.b != other.b:
            raise TypeError(f"Points {self}, {other} are not on the same curve")
        
        if self.x is None: # self is the point at infinity
            return other
        elif other.x is None: # other is the point at infinity
            return self

        if self.x == other.x and self.y != other.y:
            return self.__get_inf()

        if self.x != other.x:
            slope = (other.y - self.y) * pow(other.x - self.x, -1, self.p)
            x3 = (slope ** 2 - self.x - other.x) % self.p
            y3 = (slope * (self.x - x3) - self.y) % self.p
            return Point(x3, y3, self.a, self.b, self.p)
        
        if self == other:
            slope = (3 * self.x ** 2 + self.a) * pow(2 * self.y, -1, self.p)
            x3 = (slope ** 2 - 2 * self.x) % self.p
            y3 = (slope * (self.x - x3) - self.y) % self.p
            return Point(x3, y3, self.a, self.b, self.p)
        
    def __neg__(self):
        return Point(self.x, -self.y % self.p, self.a, self.b, self.p)

    def __sub__(self, other):
        return self + -other

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.a == other.a and self.b == other.b and self.p == other.p
    
    def __rmul__(self, coefficient):
        coef = coefficient
        current = self
        result = self.__get_inf()
        
        while coef:
            if coef & 1:
                result += current
                
            current += current
            coef >>= 1

        return result

In [3]:
# Elliptic curve equation: y^2 = x^3 + ax + b
a = -1
b = 1
p = 97

In [4]:
# Initialize points P and Q
P = Point(17, 85, a, b, p)
Q = Point(58, 52, a, b, p)
print(P)
print(Q)

Point(17, 85) on curve y^2 = x^3 + 96x + 1 mod 97
Point(58, 52) on curve y^2 = x^3 + 96x + 1 mod 97


In [5]:
# Encryption: C = P + kQ (k is an integer)
k = 20  # Private key
C = P
for _ in range(k):
    C += Q

print("Encrypted Point:", C.x, C.y)

Encrypted Point: 85 15


In [6]:
# Decryption: P = C - kQ
D = C
for _ in range(k):
    D -= Q

print("Decrypted Point:", D.x, D.y)

# Now, D should be the same as the original point P.
assert D == P

Decrypted Point: 17 85
